<a href="https://colab.research.google.com/github/tanish979/Celebal/blob/main/Road_Sign_Detection_System_using_Yolov8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
if 'COLAB_GPU'in os.environ:
  print("enabled")
else:
  print("disabled")

enabled


In [ ]:
import sys
print(sys.version)


3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]


In [ ]:
# Install roboflow and ultralytics libraries
%pip install roboflow ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 113.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 115.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.

In [ ]:
import ultralytics
ultralytics.checks()

Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 42.1/112.6 GB disk)


In [ ]:
%pwd

'/content'

In [ ]:
# Import necessary libraries
from roboflow import Roboflow
from ultralytics import YOLO
import os

In [ ]:

# Download the Self-Driving Cars dataset from Roboflow
# Replace 'YOUR_ROBOFLOW_API_KEY' with your actual Roboflow API key
rf = Roboflow(api_key="hxYBamzDLGHyW2mx2o9N")
project = rf.workspace("selfdriving-car-qtywx").project("self-driving-cars-lfjou")
dataset = project.version(1).download("yolov8")  # Downloads in YOLOv8 format

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Self-Driving-Cars-1 in yolov8:: 100%|██████████| 8660/8660 [00:00<00:00, 8861.88it/s]


In [ ]:
# Create data.yaml with provided configuration
dataset_yaml = """
names:
- Green Light
- Red Light
- Speed Limit 100
- Speed Limit 110
- Speed Limit 120
- Speed Limit 20
- Speed Limit 30
- Speed Limit 40

- Speed Limit 50
- Speed Limit 60
- Speed Limit 70
- Speed Limit 80
- Speed Limit 90
- Stop
nc: 14
test: '/home/linux/Projects/road_sign_detection_using_Yolo8/Self-Driving-Cars-1/test/images'
train: '/home/linux/Projects/road_sign_detection_using_Yolo8/Self-Driving-Cars-1/train/images'
val: '/home/linux/Projects/road_sign_detection_using_Yolo8/Self-Driving-Cars-1/valid/images'
"""

# Save the YAML file
yaml_path = "/content/Self-Driving-Cars-1/data.yaml"
with open(yaml_path, "w") as f:
    f.write(dataset_yaml)

In [ ]:
# Load the YOLOv8 nano model
model = YOLO("yolov8n.pt")  # Use nano model for lightweight training

100%|██████████| 6.25M/6.25M [00:00<00:00, 120MB/s]


In [ ]:
import os

dataset_path = "/content/Self-Driving-Cars-1"
for split in ["train", "valid", "test"]:
    images_dir = os.path.join(dataset_path, split, "images")
    labels_dir = os.path.join(dataset_path, split, "labels")
    if os.path.exists(images_dir) and os.path.exists(labels_dir):
        image_files = {f.replace('.jpg', '').replace('.png', '') for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png'))}
        label_files = {f.replace('.txt', '') for f in os.listdir(labels_dir) if f.endswith('.txt')}
        missing_images = label_files - image_files
        missing_labels = image_files - label_files
        print(f"{split} - Missing images for labels: {missing_images}")
        print(f"{split} - Missing labels for images: {missing_labels}")
        print(f"{split} - Image count: {len(image_files)}, Label count: {len(label_files)}")
    else:
        print(f"{split} directory missing or incomplete")

train - Missing images for labels: set()
train - Missing labels for images: set()
train - Image count: 3022, Label count: 3022
valid - Missing images for labels: set()
valid - Missing labels for images: set()
valid - Image count: 728, Label count: 728
test - Missing images for labels: set()
test - Missing labels for images: set()
test - Image count: 574, Label count: 574


In [ ]:
model.train(
    data="/content/Self-Driving-Cars-1/data.yaml",  # Your dataset path
    epochs=10,                   # More epochs for better training
    imgsz=640,                   # Standard size for YOLOv8
    batch=16,                    # Try 16 or 32 (depending on RAM/GPU)
    workers=2,                   # Safe for Colab
    cache=True,                  # Speeds up training after first epoch
    name="yolov8_colab_gpu_run", # Run name
    device=0,                    # Use GPU (0 = first GPU)
    lr0=0.001,                   # Learning rate
    warmup_epochs=3.0,           # Optional, helps stabilize early training
    val=True,                    # Enable validation
    plots=True                   # Enable plots (optional)
)


Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Self-Driving-Cars-1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8_colab_gpu_run, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True,

100%|██████████| 755k/755k [00:00<00:00, 20.6MB/s]

Overriding model.yaml nc=80 with nc=14

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

Model summary: 129 layers, 3,013,578 parameters, 3,013,562 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|██████████| 5.35M/5.35M [00:00<00:00, 111MB/s]


...

                   all        728        811      0.955      0.897      0.965      0.846



10 epochs completed in 0.140 hours.
Optimizer stripped from runs/detect/yolov8_colab_gpu_run/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/yolov8_colab_gpu_run/weights/best.pt, 6.2MB

Validating runs/detect/yolov8_colab_gpu_run/weights/best.pt...
Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,008,378 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:07<00:00,  2.97it/s]


                   all        728        811      0.955      0.898      0.965      0.845
           Green Light         59         84      0.862      0.744      0.854      0.522
             Red Light         48         73      0.829      0.733      0.866      0.549
       Speed Limit 100         49         49      0.923          1      0.993      0.908
       Speed Limit 110         15         15          1       0.78      0.982      0.918
       Speed Limit 120         55         55      0.963      0.943      0.984      0.905
        Speed Limit 20         50         50       0.99          1      0.995      0.885
        Speed Limit 30         70         73       0.97      0.973      0.982      0.914
        Speed Limit 40         51         52      0.995      0.962      0.989       0.88
        Speed Limit 50         63         64          1      0.792      0.966      0.878
        Speed Limit 60         69         69       0.97      0.957      0.983      0.895
        Speed Limit 7

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d7a420166d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.0

In [ ]:
# Perform inference on test images
results = model.predict(
    source="/content/Self-Driving-Cars-1/test/images",
    save=True,                            # Save predictions to runs/detect/predict/
    conf=0.5                              # Confidence threshold
)


image 1/574 /content/Self-Driving-Cars-1/test/images/000003_jpg.rf.ff818a438b1c75ce3c92d15267428783.jpg: 640x640 1 Speed Limit 30, 9.1ms
image 2/574 /content/Self-Driving-Cars-1/test/images/000006_jpg.rf.6e24bb43ad0056bc30e939f64f62fc8c.jpg: 640x640 1 Speed Limit 30, 7.4ms
image 3/574 /content/Self-Driving-Cars-1/test/images/00000_00000_00003_png.rf.dc4a7394271078f3ee800bf899464949.jpg: 640x640 1 Speed Limit 20, 7.4ms
image 4/574 /content/Self-Driving-Cars-1/test/images/00000_00000_00005_png.rf.533b7a9c3eb656232d7f23dc09c66621.jpg: 640x640 1 Speed Limit 20, 7.4ms
image 5/574 /content/Self-Driving-Cars-1/test/images/00000_00000_00006_png.rf.85f7b0aaafdbf34eeef04d99046643ee.jpg: 640x640 1 Speed Limit 20, 7.4ms
image 6/574 /content/Self-Driving-Cars-1/test/images/00000_00000_00008_png.rf.52af86c353cb1b55312663b67c17b9fc.jpg: 640x640 1 Speed Limit 20, 7.4ms
image 7/574 /content/Self-Driving-Cars-1/test/images/00000_00000_00013_png.rf.2dd84e650965dc8aefe237c2f003e7ee.jpg: 640x640 1 Speed L

In [ ]:
# Evaluate the model on the validation set
metrics = model.val()
print("Validation Metrics:")
print(f"mAP@50: {metrics.box.map50:.3f}")
print(f"mAP@50:95: {metrics.box.map:.3f}")

Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 646.0±215.7 MB/s, size: 19.0 KB)


val: Scanning /content/Self-Driving-Cars-1/valid/labels.cache... 728 images, 0 backgrounds, 0 corrupt: 100%|██████████| 728/728 [00:00<?, ?it/s]

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.



val: Caching images (0.8GB RAM): 100%|██████████| 728/728 [00:01<00:00, 390.78it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 46/46 [00:08<00:00,  5.41it/s]


                   all        728        811      0.957      0.897      0.965      0.845
           Green Light         59         84      0.862      0.742      0.854      0.517
             Red Light         48         73      0.855      0.726      0.867       0.55
       Speed Limit 100         49         49      0.924          1      0.992      0.905
       Speed Limit 110         15         15          1      0.778      0.978      0.914
       Speed Limit 120         55         55      0.963      0.942      0.984      0.906
        Speed Limit 20         50         50       0.99          1      0.995      0.884
        Speed Limit 30         70         73      0.973      0.973      0.982      0.914
        Speed Limit 40         51         52      0.995      0.962      0.989       0.88
        Speed Limit 50         63         64          1      0.792      0.967      0.883
        Speed Limit 60         69         69       0.97      0.957      0.983      0.895
        Speed Limit 7

In [ ]:
# Save the trained model
model.save("self_driving_cars_yolov8.pt")

In [ ]:
from ultralytics import YOLO

model = YOLO("self_driving_cars_yolov8.pt")


In [ ]:
results = model.predict(source="/content/Self-Driving-Cars-1/test/images", save=True, conf=0.5)



image 1/574 /content/Self-Driving-Cars-1/test/images/000003_jpg.rf.ff818a438b1c75ce3c92d15267428783.jpg: 640x640 1 Speed Limit 30, 8.0ms
image 2/574 /content/Self-Driving-Cars-1/test/images/000006_jpg.rf.6e24bb43ad0056bc30e939f64f62fc8c.jpg: 640x640 1 Speed Limit 30, 7.4ms
image 3/574 /content/Self-Driving-Cars-1/test/images/00000_00000_00003_png.rf.dc4a7394271078f3ee800bf899464949.jpg: 640x640 1 Speed Limit 20, 7.4ms
image 4/574 /content/Self-Driving-Cars-1/test/images/00000_00000_00005_png.rf.533b7a9c3eb656232d7f23dc09c66621.jpg: 640x640 1 Speed Limit 20, 7.4ms
image 5/574 /content/Self-Driving-Cars-1/test/images/00000_00000_00006_png.rf.85f7b0aaafdbf34eeef04d99046643ee.jpg: 640x640 1 Speed Limit 20, 7.4ms
image 6/574 /content/Self-Driving-Cars-1/test/images/00000_00000_00008_png.rf.52af86c353cb1b55312663b67c17b9fc.jpg: 640x640 1 Speed Limit 20, 7.4ms
image 7/574 /content/Self-Driving-Cars-1/test/images/00000_00000_00013_png.rf.2dd84e650965dc8aefe237c2f003e7ee.jpg: 640x640 1 Speed L

In [30]:
from google.colab import files
uploaded = files.upload()


Saving traffic signal images and videos - Yahoo India Image Search results.html to traffic signal images and videos - Yahoo India Image Search results.html


In [33]:
from ultralytics import YOLO

# Load your trained model
model = YOLO("self_driving_cars_yolov8.pt")

# Run prediction on the uploaded video
results = model.predict(source="22169_tiny.mp4", save=True)


FileNotFoundError: No images or videos found in traffic signal images and videos - Yahoo India Image Search results.html. Supported formats are:
images: {'tiff', 'webp', 'heic', 'jpg', 'tif', 'bmp', 'pfm', 'dng', 'jpeg', 'png', 'mpo'}
videos: {'ts', 'mov', 'gif', 'avi', 'mpg', 'm4v', 'mp4', 'webm', 'mpeg', 'mkv', 'wmv', 'asf'}